[![Binder](https://mybinder.org/badge_logo.svg)](https://notebooks.gesis.org/binder/v2/gh/jolin-io/fall-in-love-with-julia-14/main?filepath=01%20Introduction%20to%20JuMP.ipynb)

<a href="https://www.jolin.io" target="_blank" rel="noreferrer noopener">
<img src="https://www.jolin.io/assets/Jolin/Jolin-Banner-Website-v1.3-darkmode.webp">
</a>

# Introduction to JuMP

JuMP.jl let's you solve mathematical optimization problems like contraint programming or schedule optimization.
Essentially it gives you a modeling language to interface numerous solvers.

In [87]:
using JuMP, HiGHS
model = Model(HiGHS.Optimizer)

A JuMP Model
Feasibility problem with:
Variables: 0
Model mode: AUTOMATIC
CachingOptimizer state: EMPTY_OPTIMIZER
Solver name: HiGHS

## The Basics

### Variables

In [88]:
# named style
@variable(model, 0 <= v1 <= 6)  # default type is Float
@variable(model, v2, Bin)  # Int or Bin (Binary) are also valid

# anonymous style
v3 = @variable(model, integer=true, lower_bound=0)  # binary keyword

_[3]

In [89]:
# named variables are stored in the model
model[:v1]

v1

In [90]:
JuMP.object_dictionary(model)

Dict{Symbol, Any} with 2 entries:
  :v2 => v2
  :v1 => v1

### Constraints

Typically `<=` comparisons. Strict comparisons need to be modeled by an epsilon difference.

In [91]:
# named constraint
@constraint(model, c1, 6v1 >= 100v2 + 3)

# anonymous constraint
c2 = @constraint(model, 7v3 >= 300(1-v2))

300 v2 + 7 _[3] ≥ 300

In [92]:
# named constraints get also listed
JuMP.object_dictionary(model)

Dict{Symbol, Any} with 3 entries:
  :v2 => v2
  :c1 => c1 : 6 v1 - 100 v2 ≥ 3
  :v1 => v1

Different solvers can deal with different complexities. `HiGHS` can solve Mixed Integer Linear Programs, i.e. for example no quadratic constraints

### Objective

The goal, how the model shall be optimized

In [93]:
@objective(model, Min, 12v1 + v2 + 20v3)

12 v1 + v2 + 20 _[3]

### Solve

In [94]:
optimize!(model)

Running HiGHS 1.6.0: Copyright (c) 2023 HiGHS under MIT licence terms
Presolving model
2 rows, 2 cols, 2 nonzeros
0 rows, 0 cols, 0 nonzeros
Presolve: Optimal

Solving report
  Status            Optimal
  Primal bound      866
  Dual bound        866
  Gap               0% (tolerance: 0.01%)
  Solution status   feasible
                    866 (objective)
                    0 (bound viol.)
                    0 (int. viol.)
                    0 (row viol.)
  Timing            0.00 (total)
                    0.00 (presolve)
                    0.00 (postsolve)
  Nodes             0
  LP iterations     0 (total)
                    0 (strong br.)
                    0 (separation)
                    0 (heuristics)


In [100]:
# Always check whether the model was actually solved
is_solved_and_feasible(model) || error("Knock. Knock. I need your help.")

true

In [101]:
# the value function can be used to extract solutions
(
    minimum = objective_value(model),
    variables = (value(v1), value(model[:v2]), value(v3)),
    constraints = (value(c1), value(c2)),
)

(minimum = 866.0, variables = (0.5, 0.0, 43.0), constraints = (3.0, 301.0))

**👉 Your Challenge:** Change the constraints/objective such that `v1` becomes the largest value.

## N-Queens and indexed variables

Adapted from a tutorial by Matthew Helm and Mathieu Tanneau.

![N-Queens illustration](https://jump.dev/JuMP.jl/stable/assets/n_queens4.png)

In [109]:
using LinearAlgebra
model_queens = Model(HiGHS.Optimizer)
N = 8

8

### Indexed Variables

This is actually super powerful - you can use arbitrary julia ranges or vectors for indices.
E.g. imagine indexing by date.

In [110]:
@variable(model_queens, x[1:N, 1:N], Bin)

8×8 Matrix{VariableRef}:
 x[1,1]  x[1,2]  x[1,3]  x[1,4]  x[1,5]  x[1,6]  x[1,7]  x[1,8]
 x[2,1]  x[2,2]  x[2,3]  x[2,4]  x[2,5]  x[2,6]  x[2,7]  x[2,8]
 x[3,1]  x[3,2]  x[3,3]  x[3,4]  x[3,5]  x[3,6]  x[3,7]  x[3,8]
 x[4,1]  x[4,2]  x[4,3]  x[4,4]  x[4,5]  x[4,6]  x[4,7]  x[4,8]
 x[5,1]  x[5,2]  x[5,3]  x[5,4]  x[5,5]  x[5,6]  x[5,7]  x[5,8]
 x[6,1]  x[6,2]  x[6,3]  x[6,4]  x[6,5]  x[6,6]  x[6,7]  x[6,8]
 x[7,1]  x[7,2]  x[7,3]  x[7,4]  x[7,5]  x[7,6]  x[7,7]  x[7,8]
 x[8,1]  x[8,2]  x[8,3]  x[8,4]  x[8,5]  x[8,6]  x[8,7]  x[8,8]

### Constraints

Similarly, constraints can be indexed. However here simple constraints are enough.

In [112]:
# There must be exactly one queen in a given row/column
for i in 1:N
    # anonymous constraints get handy inside loops
    @constraint(model_queens, sum(x[i, :]) == 1)
    @constraint(model_queens, sum(x[:, i]) == 1)
end

In [114]:
# There can only be one queen on any given diagonal
for i in -(N - 1):(N-1)
    # diag comes from LinearAlgebra
    @constraint(model_queens, sum(diag(x, i)) <= 1)
    @constraint(model_queens, sum(diag(reverse(x; dims = 1), i)) <= 1)
end

### Solve

This is how to combine JuMP model with data.

In [116]:
fix(x[1, 2], 1)
fix(x[5, 3], 1)

now optimize given these given queen positions

In [131]:
set_attribute(model_queens, "output_flag", false)  # hide HiGHS optimize! output 
optimize!(model_queens)
@assert is_solved_and_feasible(model_queens)
round.(Int, value.(x))

8×8 Matrix{Int64}:
 0  1  0  0  0  0  0  0
 0  0  0  1  0  0  0  0
 0  0  0  0  0  1  0  0
 0  0  0  0  0  0  0  1
 0  0  1  0  0  0  0  0
 1  0  0  0  0  0  0  0
 0  0  0  0  0  0  1  0
 0  0  0  0  1  0  0  0

**👉 Your Challenge:** Change the initial fixed queens. What happens if you introduce an infeasible situation?

## Sudoku and constraint programming

Adapted from a tutorial by Iain Dunning.

![Sudoku illustration](https://jump.dev/JuMP.jl/stable/assets/partial_sudoku.png)

You can model Sudoku similar to N-Queens with Binary variables. Or you use contraint programming.


In [135]:
model_sudoku = Model(HiGHS.Optimizer)

A JuMP Model
Feasibility problem with:
Variables: 0
Model mode: AUTOMATIC
CachingOptimizer state: EMPTY_OPTIMIZER
Solver name: HiGHS

Instead of the binary variables, we directly define a 9x9 grid of integer values between 1 and 9:

In [137]:
@variable(model_sudoku, 1 <= x[1:9, 1:9] <= 9, Int)

9×9 Matrix{VariableRef}:
 x[1,1]  x[1,2]  x[1,3]  x[1,4]  x[1,5]  x[1,6]  x[1,7]  x[1,8]  x[1,9]
 x[2,1]  x[2,2]  x[2,3]  x[2,4]  x[2,5]  x[2,6]  x[2,7]  x[2,8]  x[2,9]
 x[3,1]  x[3,2]  x[3,3]  x[3,4]  x[3,5]  x[3,6]  x[3,7]  x[3,8]  x[3,9]
 x[4,1]  x[4,2]  x[4,3]  x[4,4]  x[4,5]  x[4,6]  x[4,7]  x[4,8]  x[4,9]
 x[5,1]  x[5,2]  x[5,3]  x[5,4]  x[5,5]  x[5,6]  x[5,7]  x[5,8]  x[5,9]
 x[6,1]  x[6,2]  x[6,3]  x[6,4]  x[6,5]  x[6,6]  x[6,7]  x[6,8]  x[6,9]
 x[7,1]  x[7,2]  x[7,3]  x[7,4]  x[7,5]  x[7,6]  x[7,7]  x[7,8]  x[7,9]
 x[8,1]  x[8,2]  x[8,3]  x[8,4]  x[8,5]  x[8,6]  x[8,7]  x[8,8]  x[8,9]
 x[9,1]  x[9,2]  x[9,3]  x[9,4]  x[9,5]  x[9,6]  x[9,7]  x[9,8]  x[9,9]

### Constraint Sets

JuMP has a couple of [special Sets](https://jump.dev/JuMP.jl/stable/tutorials/linear/constraint_programming/) which can be used for constraint programming.

In [144]:
# values in each row/column must be all-different
# note, these are indexed constraints, a bit like a for loop
@constraint(model_sudoku, [i = 1:9], x[i, :] in MOI.AllDifferent(9))
@constraint(model_sudoku, [j = 1:9], x[:, j] in MOI.AllDifferent(9));

In [141]:
# values in each 3x3 sub-grid must be all-different
for i in (0, 3, 6), j in (0, 3, 6)
    @constraint(model_sudoku, vec(x[i.+(1:3), j.+(1:3)]) in MOI.AllDifferent(9))
end

### Solve

In [147]:
set_attribute(model_sudoku, "output_flag", false)
optimize!(model_sudoku)
@assert is_solved_and_feasible(model_sudoku)
round.(Int, value.(x))

9×9 Matrix{Int64}:
 9  7  5  8  2  3  1  4  6
 1  6  3  9  7  4  8  5  2
 8  2  4  6  1  5  9  3  7
 7  3  8  2  5  6  4  9  1
 4  1  6  7  9  8  5  2  3
 2  5  9  3  4  1  7  6  8
 6  4  7  1  3  9  2  8  5
 5  8  1  4  6  2  3  7  9
 3  9  2  5  8  7  6  1  4

### Fix initial values

**👉 Your Challenge:** Similarly to N-Queens, fix the given numbers for the Sudoku example.

In [149]:
# Your Space
# ...

# Next: [02 Tips and Tricks and DisjunctiveProgramming](https://notebooks.gesis.org/binder/v2/gh/jolin-io/fall-in-love-with-julia-14/main?filepath=02%20Tips%20and%20Tricks%20and%20DisjunctiveProgramming.ipynb)


For questions or suggestions please contact me at stephan.sahm@jolin.io

<a href="https://www.jolin.io" target="_blank" rel="noreferrer noopener">
<img src="https://www.jolin.io/assets/Jolin/Jolin-Banner-Website-v1.3-darkmode.webp">
</a>
